In [81]:
import numpy as np


In [82]:
cube = np.array([[0,0,1],[0,1,1],[1,1,1],[1,0,1],[0,0,0],[0,1,0],[1,1,0],[1,0,0]])

In [83]:
list1 = []

In [84]:
def two_point(cord1,cord2):
    
    x_dir = cord2[0] - cord1[0]
    y_dir = cord2[1] - cord1[1]
    z_dir = cord2[2] - cord1[2]

    # equation = "<x,y,z>"+" = "+"<"+str(cord1[0])+","+str(cord1[1])+","+str(cord1[2])+">"+" + "+"t*("+str(x_dir)+","+str(y_dir)+","+str(z_dir)+")"
    # return equation
    return [[x_dir,y_dir,z_dir],cord1]

In [56]:
# list1.append(two_point([4,0,4],[4,4,4]))
# list1.append(two_point([0,0,4],[0,4,4]))

In [85]:
list1 = [[[12,12,11], [26,-17,10]], [[321,-61,2], [-51,-61,-52]]]

In [86]:
def line_test(dataset):
    
    # parallel
    flag = 0
    if flag == 0:
        i = 0
        while i<100:
                if list1[0][0] == list(np.dot(i,list1[1][0])):
                    flag = 1
                    print("The Lines are parallel")
                    break
                else:
                    i += 1
    
    # Intersecting
    if flag == 0:

        
        value1 = list1[1][1][0] - list1[0][1][0]
        value2 = list1[1][1][1] - list1[0][1][1]
        B = np.array([[value1],[value2]])
        A = np.array([[list1[0][0][0],-list1[1][0][0]],[list1[0][0][1],-list1[1][0][1]]])
        try:
            inv_A = np.linalg.inv(A)
        except :
            print("The Line is parallel (Matrix is singular)")
        else:
            X = np.dot(inv_A,B)
            print("The lines are intersecting at"+str(int(X[0]))+","+str(int(X[1])))
            flag = 1

    #Skew
    if flag == 0:
        flag = 1
        print("The lines are drifting apart")

    return A,B,X,inv_A


In [87]:
A,B,X,inv_A = line_test(list1)

The lines are intersecting at-4,0
